In [1]:
from faker import Faker
import json
import sqlite3
import random
import secrets

In [2]:
# gacha function to generate employee role
def gacha_role():
    role = [
        "Manager",
        "Assistant Manager",
        "Supervisor",
        "Accountant",
        "Clerk",
        "Salesman",
        "Driver",
        "Storkeeper"
    ]
    
    weights = [
        0.05,  # Manager
        0.10,  # Assistant Manager
        0.15,  # Supervisor
        0.20,  # Accountant
        0.20,  # Clerk
        0.15,  # Salesman
        0.10,  # Driver
        0.05   # Storkeeper
    ]
    
    selected_role = random.choices(role, weights, k=1)[0]
    
    return selected_role

# gacha gender
def gacha_gender():
    gender = ['Male', 'Female']
    weights = [0.5, 0.5]
    
    return random.choices(gender, weights, k=1)[0]

In [3]:
# cyrpto id generator
def crypto_id():
    return secrets.token_hex(36)

In [4]:
# fake employee data generator
def fake_employee():
    fake = Faker(['en_US', 'ja_JP', 'ko_KR', 'zh_CN'])
    
    fake_employee = {
        "employee_id": crypto_id(),
        "age": fake.random_int(min=18, max=65),
        "name" : fake.name(),
        "gender": gacha_gender(),
        "address" : fake.address(),
        "role": gacha_role(),
        "email": fake.email(),
        "phone_number": fake.phone_number(),
        "salary": fake.random_int(min=1000, max=10000),
        "hire_date": fake.date(),
    }
    
    return fake_employee

# generate multiple fake employee JSON
def generate_multiple_fake_employee(n):
    employees = [fake_employee() for _ in range(n)]
    return json.dumps(employees)

In [5]:
print(fake_employee())

{'employee_id': 'b36e5b143ea3453123dff7f7975cf0a2d6abd2e1179ab2eacd16fc965e8db19d6ce864a0', 'age': 63, 'name': '苗婷', 'gender': 'Female', 'address': '부산광역시 강북구 테헤란6로', 'role': 'Salesman', 'email': 'dgang@example.net', 'phone_number': '090-9057-2248', 'salary': 8080, 'hire_date': '2004-12-05'}


In [6]:
# insert fake employee data to sqlite3
conn = sqlite3.connect('database/database.db')
c = conn.cursor()

c.execute(
    '''
    CREATE TABLE IF NOT EXISTS employees (
        employee_id TEXT PRIMARY KEY NOT NULL,
        age INTEGER NOT NULL,
        name TEXT NOT NULL,
        gender TEXT NOT NULL,
        address TEXT NOT NULL,
        role TEXT NOT NULL,
        email TEXT NOT NULL,
        phone_number TEXT NOT NULL,
        salary REAL NOT NULL,
        hire_date TEXT NOT NULL
    )
    '''
)

json_data = generate_multiple_fake_employee(200)

# save the json data as json
with open('json_data/employees.json', 'w') as f:
    f.write(json_data)

for employee in json.loads(json_data):
    c.execute(
        '''INSERT INTO employees(employee_id, age, name,gender,address,role, email, phone_number, salary, hire_date)
            VALUES(?,?,?,?,?,?,?,?,?,?)
        ''', 
            (
            employee['employee_id'], 
            employee['age'],
            employee['name'],
            employee['gender'],
            employee['address'],
            employee['role'],
            employee['email'],
            employee['phone_number'],
            employee['salary'],
            employee['hire_date']
            )
    )
                
conn.commit()
conn.close()
                 

In [7]:
# delete the table employees
#conn = sqlite3.connect('database/database.db')
#c = conn.cursor()
# c.execute('DROP TABLE employees')